# 預處理思路
從驗證的public dataset中可以看到需要從日期、站點ID、時間資料去進行每20分鐘的可用腳踏車數量的預測，因此訓練資料集必須整理成含有以上資訊的資料集，因為保持數據集可處理的彈性，以下程式碼我會分階段進行，主要是把經緯度的資料加進去並去除無效資料，但我還是有讓處理過程都輸出成csv檔案，因為驗證的公式需要tot(停車柱的數量)的資料，最後的train_data有兩種主要是保留時序的彈性。

訓練資料集處理後的特徵包含date, station_id, time, latitude, longitude希望用這五個資料去預測sbi，因此驗證數據集也需要整理成這格式，看之後模型要怎麼處理這些特徵值。


## ENV Check

In [ ]:
!nvidia-smi

In [ ]:
! pwd

## Data Processing and Feature Acquisition
* 輸出檔名為aggregated_data_YYYYMMDD的csv檔案到根目錄，資料很多會花一些時間
* 輸出完成後我手動在根目錄新建一個名為"aggreated_data"的資料夾把這些csv檔案都放進去 (我懶得用程式)

In [ ]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta

def convert_time_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

def process_json_file(json_path, site_id):
    with open(json_path, 'r') as file:
        data = json.load(file)

    # Get the sorted list of times from the data
    sorted_times = sorted(data.keys(), key=lambda x: convert_time_to_minutes(x))

    # Find the first non-empty data entry
    for time in sorted_times:
        if data[time]:
            last_valid_data = data[time]
            break
    else:
        last_valid_data = {'tot': 0, 'sbi': 0, 'bemp': 0, 'act': '0'}

    # Process data for every 20 minutes interval
    processed_data = []
    for minutes in range(0, 24 * 60, 20):
        current_time_str = f"{minutes // 60:02d}:{minutes % 60:02d}"
        if current_time_str in data and data[current_time_str]:
            last_valid_data = data[current_time_str]

        processed_data.append({
            'station_id': site_id,
            'Time (minutes)': minutes,
            'tot': last_valid_data.get('tot', 0),
            'sbi': last_valid_data.get('sbi', 0),
            'bemp': last_valid_data.get('bemp', 0),
            'act': last_valid_data.get('act', '0')
        })

    return processed_data

def main():
    base_path = '/data/html.2023.final.data-release/release/'
    start_date = datetime(2023, 10, 2)
    end_date = datetime(2023, 12, 8)

    current_date = start_date
    while current_date <= end_date:
        folder_name = current_date.strftime('%Y%m%d')
        folder_path = os.path.join(base_path, folder_name)
        if os.path.exists(folder_path):
            all_data = []
            for file_name in os.listdir(folder_path):
                if file_name.endswith('.json'):
                    json_path = os.path.join(folder_path, file_name)
                    # Extract site ID from file name
                    site_id = file_name.split('.')[0]
                    all_data.extend(process_json_file(json_path, site_id))

            # Convert the aggregated data into a DataFrame
            if all_data:
                df = pd.DataFrame(all_data)
                output_csv = f'/data/aggregated_data_{folder_name}.csv'
                df.to_csv(output_csv, index=False)
        current_date += timedelta(days=1)

if __name__ == '__main__':
    main()


## Capture latitude and longitude data from demographic
* 提取demographic檔案經緯度資料並輸出csv檔案

In [ ]:
# The previous path used was incorrect. Let's correct the file path.
demographic_file_path = '/data/html.2023.final.data-release/demographic.json'

# Function to extract station ID, latitude, and longitude and output it as a CSV
def extract_station_info_to_csv(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Prepare data for DataFrame
    stations_data = [{
        'station_id': sno,
        'latitude': info['lat'],
        'longitude': info['lng']
    } for sno, info in data.items()]

    # Convert to DataFrame
    df = pd.DataFrame(stations_data)
    
    # Define the CSV output path
    output_csv_path = '/data/station_info.csv'
    
    # Save as CSV
    df.to_csv(output_csv_path, index=False)
    
    return output_csv_path

# Call the function and get the path of the created CSV
csv_output_path = extract_station_info_to_csv(demographic_file_path)
csv_output_path


## Batch aggreated data and merge date
* 批次處理"aggreated_data"資料夾內的檔案並與經緯度合併 > 最後輸出一個包含所有資料的csv資料集

In [ ]:
import os
import pandas as pd

# Define the base directory containing the aggregated data files
base_dir = '/data/aggreated_data/'

# Load station information
station_info_file_path = '/data/station_info.csv'
station_info_df = pd.read_csv(station_info_file_path)
station_info_df['station_id'] = station_info_df['station_id'].astype(int)

# Initialize an empty DataFrame to store the merged data
all_data_merged = pd.DataFrame()

# Iterate through each file in the directory
for file_name in os.listdir(base_dir):
    if file_name.startswith('aggregated_data_') and file_name.endswith('.csv'):
        # Extract the date from the filename
        date_str = file_name[len('aggregated_data_'):-4]
        file_path = os.path.join(base_dir, file_name)

        # Load the aggregated data file
        aggregated_data_df = pd.read_csv(file_path)
        aggregated_data_df['station_id'] = aggregated_data_df['station_id'].astype(int)

        # Add the extracted date to the DataFrame
        # Convert date to the desired format without separators (YYYYMMDD)
        aggregated_data_df['date'] = pd.to_datetime(date_str).strftime('%Y%m%d')

        # Merge with station information
        merged_df = pd.merge(aggregated_data_df, station_info_df, how='left', on='station_id')

        # Append to the overall DataFrame
        all_data_merged = pd.concat([all_data_merged, merged_df])

# Reset the index of the final DataFrame
all_data_merged.reset_index(drop=True, inplace=True)

# Save the final merged data to a new CSV file
output_file_path = '/data/merged_all_data.csv'
all_data_merged.to_csv(output_file_path, index=False)

# Output file path for download
output_file_path



## Sort columns
* 單純看不順眼sort一下

In [ ]:
# Re-import the necessary libraries
import pandas as pd

# Load the final merged dataset again due to the environment reset
final_merged_dataset_path = '/data/merged_all_data.csv'
final_merged_dataset = pd.read_csv(final_merged_dataset_path)

# Reorder the columns according to the new requirement
final_merged_dataset = final_merged_dataset[['date', 'station_id', 'Time (minutes)', 'latitude', 'longitude', 'tot', 'sbi', 'bemp', 'act']]

# Rename the columns for consistency
final_merged_dataset.rename(columns={'Time (minutes)': 'time'}, inplace=True)

# Sort the DataFrame by 'date' and 'station_id'
final_merged_dataset.sort_values(by=['date', 'station_id'], inplace=True)

# Save the sorted and reordered DataFrame to a new CSV file
sorted_reordered_csv_path = '/data/merged_all_data_sort.csv'
final_merged_dataset.to_csv(sorted_reordered_csv_path, index=False)

sorted_reordered_csv_path


## Remove tot, bemp, act
* 去除三項值用以符合驗證資料集

In [ ]:
# Load the dataset
file_path = '/data/merged_all_data_sort.csv'  # Replace this with your file path
data = pd.read_csv(file_path)

# Remove specified columns
data_cleaned = data.drop(['tot', 'bemp', 'act'], axis=1)

# Save the cleaned dataset
data_cleaned.to_csv('train_data.csv', index=False)

## Convert minute into 00:00 (optional)
* 將時間轉換成24小時制

In [ ]:
# Load the dataset
file_path = '/data/train_data.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Function to convert time format
def convert_time(time):
    hours = time // 60
    minutes = time % 60
    return f"{hours:02d}:{minutes:02d}"

# Apply the conversion to the 'time' column
data['time'] = data['time'].apply(convert_time)

# The 'time' column is now in the desired format
# You can now work with this updated dataframe or save it to a new file
# For example, to save:
data.to_csv('train_data_time_convert.csv', index=False)
